In [1]:
import json

from pathlib import Path
from IPython.display import HTML, display

from joint import data

# put there the path to the results you want to show
FOLDER_WEIGHTED_EMB='joint/results/viz/eval_loss_both_slottype_full_we_large_recurrent_cell_lstm_attention_both_three_stages_intentextraction_word-emb___hyper:LABEL_EMB_SIZE=64,LSTM_SIZE=128,BATCH_SIZE=2,MAX_EPOCHS=100/huric_eb/modern'
FOLDER_BI_RNN='joint/results/viz/eval_loss_both_slottype_full_we_large_recurrent_cell_lstm_attention_both_three_stages___hyper:LABEL_EMB_SIZE=64,LSTM_SIZE=128,BATCH_SIZE=2,MAX_EPOCHS=100/huric_eb/modern'
FOLDER='joint/results/eval_loss_both_slottype_full_we_large_recurrent_cell_lstm_attention_both_three_stages_intentextraction_word-emb___hyper:LABEL_EMB_SIZE=64,LSTM_SIZE=128,BATCH_SIZE=2,MAX_EPOCHS=1/huric_eb/modern'

def load_json(folder, epoch=99):
    json_location = Path(folder) / 'json' / 'epoch_{}'.format(epoch) / 'prediction_fold_full.json'
    with open(json_location) as f:
        content = json.load(f)
    return content['samples']

def get_intent_attention_arrays(sentence_data):
    """returns true (lexical units) and pred attentions"""
    lexical_units = [0] * sentence_data['length']
    for lu in sentence_data['lexical_unit_ids']:
        lexical_units[lu - sentence_data['start_token_id']] = 1
    return lexical_units, sentence_data['intent_attentions']

#def get_bd_attention_arrays(sentence_data):
#    return sentence_data['words'], sentence_data['slots_pred']
   
def get_color(value):
    """Colors a cell by the value provided [0,1] if value is number, otherwise white"""
    
    if isinstance(value, str):
        return 'rgb(255,255,255)'
        print('str', value)
    else:
        v = "%.4f" %  (255 - value * 255)
        return 'rgb(255,{},{})'.format(v, v)
    
def float_to_str(value):
    if isinstance(value, str):
        return value
    else:
        return "%.4f" % float(value)
    
def display_intent(intent_attention_data):
    html_str = '<table><tr>{}</tr></table>'.format(
        '</tr><tr>'.join(
            #'<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in intent_attention_data)
            ''.join(['<td style="background-color: {};">{}</td>'.format(get_color(value), float_to_str(value)) for value in row])
            for row in intent_attention_data
        )
    )
    display(HTML(html_str))

def display_align(row_labels, col_labels, values):
    html_str = '<table><tr>{}</tr><tr>{}</tr></table>'.format(
        # the headers
        ''.join(['<td>{}</td>'.format(t) for t in [''] + row_labels]),
        # the body
        '</tr><tr>'.join(
            '<td>{}</td>'.format(col_labels[row_idx]) +
            ''.join(['<td style="background-color: {};">{}</td>'.format(get_color(value), float_to_str(value)) for value in row])
            for row_idx, row in enumerate(values)
        )
    )
    display(HTML(html_str))

def display_all(samples):
    for idx, sample in enumerate(samples):
        attentions_true, attentions_pred = get_intent_attention_arrays(sample)
        display_intent([sample['words'], attentions_true, attentions_pred])
        bd = data.slots_to_iob_only(sample['slots_pred'])
        ac = data.slots_to_types_only(sample['slots_pred'])
        display_align(sample['words'], bd, sample['bd_attentions'])
        display_align(bd, ac, sample['ac_attentions'])
        display_align(sample['words'], sample['slots_pred'], sample['slots_attentions'])
        # don't make the browser crash with all those tables
        if idx >= 10:
            return


In [2]:
data_weighted_emb = load_json(FOLDER, 0)
display_all(data_weighted_emb)

can,you,put,the,detergent,in,the,washing,machine
0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
0.0398,0.0391,0.0335,0.0459,0.0244,0.0551,0.0459,0.0457,0.1186


,can,you,put,the,detergent,in,the,washing,machine
O,0.0621,0.1548,0.4343,0.1075,0.1241,0.1033,0.0106,0.0024,0.0008
B-_,0.0904,0.2026,0.4083,0.0936,0.1097,0.0784,0.0119,0.0036,0.0015
O,0.0759,0.1775,0.4229,0.1010,0.1182,0.0894,0.0111,0.0029,0.0011
B-_,0.0946,0.1940,0.3664,0.1095,0.1236,0.0904,0.0145,0.0048,0.0021
I-_,0.0689,0.1365,0.2979,0.1522,0.1518,0.1545,0.0266,0.0082,0.0034
B-_,0.0500,0.1008,0.2743,0.1693,0.1562,0.2061,0.0325,0.0078,0.0030
I-_,0.0455,0.0883,0.2524,0.1740,0.1587,0.2298,0.0389,0.0090,0.0035
I-_,0.0444,0.0837,0.2434,0.1740,0.1595,0.2399,0.0417,0.0096,0.0039
I-_,0.0447,0.0868,0.2652,0.1682,0.1580,0.2313,0.0350,0.0076,0.0031


,O,B-_,O,B-_,I-_,B-_,I-_,I-_,I-_
O,0.0281,0.1024,0.1429,0.0705,0.1933,0.1590,0.0927,0.1417,0.0693
Agent,0.0378,0.1174,0.1444,0.0746,0.1690,0.1448,0.0920,0.1382,0.0818
O,0.0312,0.1064,0.1451,0.0725,0.1863,0.1558,0.0924,0.1380,0.0723
Theme,0.0574,0.1237,0.1346,0.0823,0.1386,0.1278,0.0963,0.1344,0.1048
Theme,0.0595,0.0927,0.1019,0.0810,0.1081,0.1162,0.1084,0.1585,0.1737
Goal,0.0582,0.1030,0.1158,0.0859,0.1241,0.1239,0.1069,0.1472,0.1349
Goal,0.0652,0.0887,0.0954,0.0827,0.1004,0.1099,0.1106,0.1579,0.1892
Goal,0.0630,0.0893,0.0970,0.0831,0.1030,0.1109,0.1105,0.1583,0.1849
Goal,0.0535,0.0876,0.0996,0.0807,0.1103,0.1165,0.1104,0.1658,0.1756


,can,you,put,the,detergent,in,the,washing,machine
O,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
B-Agent,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
O,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
B-Theme,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
I-Theme,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
B-Goal,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
I-Goal,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
I-Goal,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
I-Goal,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


go,to,living,room
1.0000,0.0000,0.0000,0.0000
0.0175,0.0216,0.0208,0.0353


...

,robot,can,you,fully,lower,the,window,blinds
O,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
O,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
B-Agent,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
O,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
B-Agent,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
B-Location,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
I-Location,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
O,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


would,you,please,bring,me,my,phone,from,the,bed,room
0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
0.0438,0.0450,0.0241,0.0139,0.0343,0.0413,0.0556,0.0254,0.0527,0.0270,0.0271


,would,you,please,bring,me,my,phone,from,the,bed,room
O,0.0052,0.0095,0.0193,0.2070,0.4269,0.1642,0.1306,0.0313,0.0054,0.0005,0.0001
B-_,0.0118,0.0214,0.0411,0.2600,0.3942,0.1283,0.1089,0.0272,0.0059,0.0009,0.0002
O,0.0077,0.0142,0.0284,0.2388,0.4111,0.1447,0.1196,0.0291,0.0055,0.0006,0.0002
O,0.0088,0.0160,0.0323,0.2467,0.4042,0.1396,0.1171,0.0286,0.0058,0.0007,0.0002
B-_,0.0184,0.0321,0.0524,0.2605,0.3608,0.1290,0.1094,0.0289,0.0068,0.0012,0.0004
B-_,0.0132,0.0214,0.0306,0.2005,0.3408,0.1850,0.1457,0.0504,0.0104,0.0015,0.0004
I-_,0.0132,0.0189,0.0236,0.1436,0.2754,0.2242,0.1765,0.0935,0.0257,0.0042,0.0011
B-_,0.0070,0.0099,0.0130,0.1088,0.2726,0.2586,0.1869,0.1092,0.0299,0.0034,0.0007
I-_,0.0066,0.0088,0.0115,0.0962,0.2553,0.2658,0.1915,0.1210,0.0381,0.0041,0.0009
I-_,0.0063,0.0081,0.0106,0.0851,0.2399,0.2739,0.1946,0.1304,0.0453,0.0048,0.0011


,O,B-_,O,O,B-_,B-_,I-_,B-_,I-_,I-_,I-_
O,0.0073,0.0140,0.0538,0.0238,0.0369,0.0540,0.2839,0.1576,0.1619,0.1653,0.0417
Agent,0.0140,0.0253,0.0790,0.0364,0.0493,0.0620,0.2317,0.1434,0.1430,0.1602,0.0558
O,0.0088,0.0164,0.0600,0.0273,0.0408,0.0570,0.2668,0.1557,0.1572,0.1642,0.0458
O,0.0093,0.0171,0.0622,0.0292,0.0430,0.0595,0.2669,0.1562,0.1544,0.1578,0.0446
Theme,0.0264,0.0409,0.0937,0.0498,0.0602,0.0692,0.1793,0.1276,0.1278,0.1524,0.0727
Theme,0.0224,0.0330,0.0750,0.0415,0.0522,0.0648,0.1736,0.1323,0.1405,0.1768,0.0878
Theme,0.0408,0.0465,0.0634,0.0439,0.0497,0.0603,0.1125,0.1069,0.1235,0.1884,0.1641
Theme,0.0304,0.0396,0.0708,0.0459,0.0554,0.0685,0.1440,0.1222,0.1342,0.1761,0.1128
Theme,0.0426,0.0474,0.0605,0.0437,0.0486,0.0585,0.1049,0.1038,0.1238,0.1906,0.1755
Theme,0.0408,0.0460,0.0607,0.0440,0.0499,0.0609,0.1103,0.1070,0.1260,0.1868,0.1676


,would,you,please,bring,me,my,phone,from,the,bed,room
O,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
B-Agent,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
O,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
O,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
B-Theme,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
B-Theme,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
I-Theme,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
B-Theme,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
I-Theme,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
I-Theme,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
